# Data preparation notebook

- Workshop: **Tutorial: High performance computing with Python/RS-DAT, EO summer school 2025**

- Date: September 3, 2025 

This is the data preparation notebook for the workshop. It retrieves a ~5GB Sentinel-2 RGB image as a geo tiff file. The dataset will be used in the workshop to demonstrate an HPC workflow.

This notebook is only for demonstration how the example data is prepared. It is not necessary to run it for preparing the data for the workshop. You can download the prepared data directly via from the [Zenodo archive](https://zenodo.org/records/16613694):

- [Download link](https://zenodo.org/api/records/16613694/files-archive) (1.6 GB)

To execute this notebook, you need extra Python packages than specified in the `environment.yml` file. Please make sure they are installed, for example, by `pip` (or other package manager):

```sh
pip install odc-stac pystac-client
```


In [ ]:
# Access satellite data
import pystac_client
import odc.stac
import rioxarray # used as an extension
from pathlib import Path

# Visualization
import matplotlib.pyplot as plt

from dask.distributed import Client, LocalCluster, Lock
import shapely

## Search for Satellite Images with STAC


In [ ]:
# Setup search client for STAC API
api_url = 'https://earth-search.aws.element84.com/v1'
client = pystac_client.Client.open(api_url)

In [ ]:
# Collection to search
collection = 'sentinel-2-c1-l2a'  # Sentinel-2, Level 2A
# Create polygon for data retrieval, assume wgs84 coordinates
coords = [[6.174316,54.438991],[4.031982,50.421694],[8.459473,50.300054],[9.865723,51.913779],[10.074463,54.215902],[6.174316,54.438991]]
polygon_data = shapely.Polygon(coords)

In [ ]:
# Setup the search
search = client.search(
    collections=[collection],
    intersects=polygon_data,
    datetime='2025-06-12/2025-06-12', # search for a single date for a consistent solar intensity
    query=['eo:cloud_cover<40']
)

# Inspect how many items match the search
search.matched()

## Open Satellite Images

In [ ]:
# Set resolution
# res = 100 # for testing
res = 20  # for data generation

# Set clipping box
bounds = (250000,5650000, 500000, 6000000)  # (minx, miny, maxx, maxy)

# Get items from the search results
items = search.item_collection()

# Load the search results as a xarray Dataset
ds = odc.stac.load(
    items,
    groupby="solar_day", # group the images within the same day
    bands=["red", "green", "blue"],
    resolution=res, # loading resolution
    chunks={'x': 2048, 'y':  2048}, # lazy loading with Dask
    dtype="uint16"
)

# Clip the dataset to the bounds
ds = ds.rio.clip_box(*bounds)

ds

In [ ]:
# Inspect the size of ds
print(f"Dataset size: {ds.nbytes / 1e6:.2f} MB")  # in MB

In [ ]:
# Simple preprocessing function to generate RGB raster
def rgb_img(ds, vmin, vmax):
    """ 
    Generate RGB raster.
    
    Sentinel-2 L2A images are provided in Digital Numbers (DNs).
    Rescaling to (0, 1] range is done by clipping the values to the provided vmin and vmax.
    Missing values are assumed to be 0.
    """
    ds_rgb = ds[["red", "green", "blue"]].to_array()
    ds_rgb = ds_rgb.clip(max=vmax, min=vmin)
    ds_rgb = (ds_rgb - vmin) / (vmax - vmin)
    
    return ds_rgb

In [ ]:
ds_rgb = rgb_img(ds.isel(time=0), vmin=200, vmax=4000)  # Select the first time step
ds_rgb = ds_rgb.rio.write_crs(ds.rio.crs)  # Write CRS to the RGB dataset
# print(f"min: {ds_rgb.min().values}, max: {ds_rgb.max().values}") # expect min: 0, max: 1
ds_rgb

## (Optional) Data investigation, SKIP WHEN GENERATING DATA

In [ ]:
# # Plot the RGB image
# fig, ax = plt.subplots(figsize=(10, 10))
# ds_rgb.plot.imshow(ax=ax)

In [ ]:
# # Investigate the data distribution
# stats = ds_rgb.data.flatten().compute()
# stats_nonzero = stats[stats > 0]
# plt.hist(stats_nonzero, bins=50)

## Write data to disk and investigate by loading as COG

See:

- [Rioxarray Dask Read/Write Examples](https://corteva.github.io/rioxarray/stable/examples/dask_read_write.html)
- [Rioxarray COG Examples](https://corteva.github.io/rioxarray/stable/examples/COG.html)


Write COG parrallelly raises warning:
```
File ./XXXX.tif has C(loud) O(ptimized) G(eoTIFF) layout. Updating it will generally result in losing part of the optimizations (but will still produce a valid GeoTIFF file). If this is acceptable, open the file with the IGNORE_COG_LAYOUT_BREAK open option set to YES.
```
See also [GDAL documentation](https://gdal.org/en/stable/drivers/raster/gtiff.html) on `IGNORE_COG_LAYOUT_BREAK`.

In [ ]:
# Write the RGB image to a GeoTIFF file parallelly
# This writes a large geo tiff
# It will not be used directly in the workshop
# But used to generate 1) a 8000x8000 COG file as AoI; 2) a ~2000x2000 cutout for training
tiff_output = f"../data/sentinel2_rgb_res_full_{res}.tif"

cluster = LocalCluster()
client = Client(cluster)
ds_rgb.rio.to_raster(
    tiff_output,
    tiled=True,
    lock=Lock("rio"),
)

client.close()

In [ ]:
# Attempt to write a COG with parallely
# This cell fail on the GDAL IGNORE_COG_LAYOUT_BREAK error

# # Write to COG format
# res=20
# tiff_output = f"../data/sentinel2_rgb_res_full_{res}.tif"
# tiff_cog_output = f"../data/sentinel2_rgb_res_full_{res}_cog.tif"

# # # Set the config, does not work
# # rasterio.env.set_gdal_config("IGNORE_COG_LAYOUT_BREAK", "YES")
# # print(rasterio.env.get_gdal_config("IGNORE_COG_LAYOUT_BREAK"))  # Should print "YES"

# cluster = LocalCluster()
# client = Client(cluster)
# ds_rgb_loaded = rioxarray.open_rasterio(tiff_output, chunks=True)
# ds_rgb_loaded.rio.to_raster(
#     tiff_cog_output, 
#     driver="COG",
#     tiled=True,
#     lock = Lock("rio")
# )
# client.close()

In [ ]:
# # check available overview levels
# import rasterio
# with rasterio.open(tiff_output) as src:
#     overview_levels = list(range(len(src.overviews(1))))
#     print(f"Available overview levels: {overview_levels}")
#     print(f"Overview factors: {src.overviews(1)}")

# ds_rgb_loaded = rioxarray.open_rasterio(file_output, overview_level=2)
# ds_rgb_loaded.plot.imshow(figsize=(8, 8))

## Make a COG ~1.5GB by cutout the full image

The 1.5GB COG us proper for demonstration purposes, considering the limited resources of the workshop environment. The full image is ~5GB and would take too long to load in the workshop.

In [ ]:
full_tiff = Path("../data/sentinel2_rgb_res_full_20.tif")
rgb_img = rioxarray.open_rasterio(full_tiff, chunks={'x': 2048, 'y': 2048})
rgb_img

In [ ]:
# Make a cutout of 8000 x 8000 pixels
rgb_img_subset = rgb_img.isel(x=slice(0, 8000), y=slice(5000, 13000))
rgb_img_subset

In [ ]:
# inspect the size of rgb_img_subset
print(f"Dataset size: {rgb_img_subset.nbytes / 1e6:.2f} MB")  # in MB

In [ ]:
# Direct COG writing persists data in memory, this takes ~1.5mins
# This is the file we will use for the workshop
rgb_img_subset.rio.to_raster(
    "../data/sentinel2_rgb_res_20_size_8000_cog.tif", driver="COG", tiled=True
)

In [ ]:
# The "windowed" option allows for writing the COG by chunks, but slow
# This will give error:
# CPLE_AppDefinedError: TIFFAppendToStrip:Maximum TIFF file size exceeded. Use BIGTIFF=YES creation option.
# which make sense, as the recommended size for tiff file on 64bit system is <4GB.
# rgb_img.rio.to_raster(
#     "../data/sentinel2_rgb_res_20_cog.tif", driver="COG", windowed=True
# )

## Make a cutout

The cutout will be used to train a segmentation model.

In [ ]:
# Write the RGB image to a GeoTIFF file parallelly
# res = 20
# tiff_output = f"../data/sentinel2_rgb_res_full_{res}.tif"

rgb = rioxarray.open_rasterio(tiff_output, chunks=True)
rgb

In [ ]:
# Inspect coordinates range of rgb
print(f"Coordinates range: {rgb.rio.bounds()}")  # Should print the bounds

In [ ]:
bounds = (263_000, 5740_000, 303_000, 5780_000)  # (minx, miny, maxx, maxy)
rgb_cutout = rgb.rio.clip_box(*bounds)
rgb_cutout

In [ ]:
rgb_cutout.plot.imshow(figsize=(8, 8), robust=True)

In [ ]:
rgb_cutout.rio.to_raster(
    f"../data/sentinel2_rgb_res_{res}_cutout.tif",
)